In [1]:
import numpy as np
from qutip import *

def depolarize(rho, p):
    """Apply depolarizing noise with probability p to state rho."""
    dim = rho.shape[0]
    return (1 - p) * rho + p * qeye(dim) / dim

# --- Define the ideal case ---

# Ideal state |+><+|
plus_state = (basis(2,0) + basis(2,1)).unit()
rho_ideal = plus_state * plus_state.dag()

# Ideal channel: S gate
S_gate = Qobj([[1,0],[0,1j]])

# Ideal measurement: X basis
plus = (basis(2,0) + basis(2,1)).unit()
minus = (basis(2,0) - basis(2,1)).unit()
M_ideal = [plus * plus.dag(), minus * minus.dag()]

In [2]:
def apply_sequence(rho, sequence):
    """Apply a sequence of unitary gates to rho."""
    rho_out = rho
    for g in sequence:
        rho_out = g * rho_out * g.dag()
    return rho_out


def failure_from_state(rho, M, fail_outcome):
    """
    Compute failure probability from a final state.

    M            : [P_plus, P_minus]
    fail_outcome : +1 -> failure is measuring +
                   -1 -> failure is measuring -
    """
    probs = [abs((m * rho).tr()) for m in M]  # [P(+), P(-)]

    if fail_outcome == +1:
        return probs[0]
    elif fail_outcome == -1:
        return probs[1]
    else:
        raise ValueError("fail_outcome must be +1 or -1")

def pfail_vector(rho, gate, M, noise=0.0):
    """
    Fixed experiment:
        ε, SS, SSSS

    Failure rules:
        ε     -> fail on "-"
        SS    -> fail on "+"
        SSSS  -> fail on "-"
    """

    sequences = [
        [],                        # ε
        [gate, gate],              # SS
        [gate, gate, gate, gate]   # SSSS
    ]

    fail_outcomes = [-1, +1, -1]

    failure_probs = []

    for seq, fail in zip(sequences, fail_outcomes):
        # Apply gates
        rho_out = apply_sequence(rho, seq)

        # Apply depolarizing noise
        if noise > 0:
            rho_out = depolarize(rho_out, noise)

        # Extract failure probability
        p_fail = failure_from_state(rho_out, M, fail)
        failure_probs.append(p_fail)
    
    return failure_probs


In [3]:
pfail = pfail_vector(rho_ideal, S_gate, M_ideal, noise=0.05)
print("Failure probabilities [ε, SS, SSSS]:", pfail)

Failure probabilities [ε, SS, SSSS]: [0.02500000000000002, 0.02500000000000002, 0.02500000000000002]


## Generating Qubit Experiments for ML

We want a small dataset of qubit experiments $(\rho, G, M)$ for a toy ML model.

- **Gauge fixing:** To avoid redundant experiments, we fix $\rho$ to be diagonal:  
$$
\rho = \begin{pmatrix} p & 0 \\ 0 & 1-p \end{pmatrix}, \quad 0 \le p \le 1
$$
This removes almost all unitary gauge freedom.  

- **State sweep:** Vary $p$ to include both pure ($p=0,1$) and mixed ($0<p<1$) states.  

- **Measurement:** Use projectors along Bloch sphere vectors:  
$$
|\psi\rangle = \cos(\theta/2)|0\rangle + e^{i\phi}\sin(\theta/2)|1\rangle
$$
Sweep $\theta$ and $\phi$ for multiple measurement directions.  

- **Gate:** Parameterize a general SU(2) unitary:  
$$
U(\alpha,\beta,\gamma) =
\begin{pmatrix}
\cos(\alpha/2) & -e^{i \gamma} \sin(\alpha/2) \\
e^{i \beta} \sin(\alpha/2) & e^{i(\beta+\gamma)} \cos(\alpha/2)
\end{pmatrix}
$$
Sweep $\alpha,\beta,\gamma$ to systematically explore gates.  

- **Dataset:** Combine all choices of $\rho, G, M$ to generate a small, non-redundant set of experiments covering pure/mixed states, gates, and measurement directions.


In [4]:
# --- 1. Generate diagonal rho (mixed + pure) ---
p_values = np.linspace(0, 1, 5)  # 5 points from pure 0/1 to mixed
rhos = [Qobj(np.diag([p, 1-p])) for p in p_values]

# --- 2. Generate measurement projectors ---
theta_values = np.linspace(0, np.pi, 3)      # 3 theta values
phi_values = np.linspace(0, 2*np.pi, 3)      # 3 phi values

measurements = []
for theta in theta_values:
    for phi in phi_values:
        ket = np.array([np.cos(theta/2), np.exp(1j*phi)*np.sin(theta/2)])
        M0 = Qobj(np.outer(ket, ket.conj()))
        M1 = Qobj(np.eye(2) - np.outer(ket, ket.conj()))
        measurements.append([M0, M1])

# --- 3. Generate SU(2) gates ---
alpha_values = np.linspace(0, np.pi, 3)
beta_values  = np.linspace(0, 2*np.pi, 3)
gamma_values = np.linspace(0, 2*np.pi, 3)

gates = []
for alpha in alpha_values:
    for beta in beta_values:
        for gamma in gamma_values:
            U = np.array([
                [np.cos(alpha/2), -np.exp(1j*gamma)*np.sin(alpha/2)],
                [np.exp(1j*beta)*np.sin(alpha/2), np.exp(1j*(beta+gamma))*np.cos(alpha/2)]
            ])
            gates.append(Qobj(U))

# --- 4. Combine into dataset ---
dataset = []
for rho in rhos:
    for M in measurements:
        for G in gates:
            dataset.append((rho, G, M))

print(f"Generated {len(dataset)} experiments")


Generated 1215 experiments


In [5]:
import numpy as np

def flatten_experiment(rho, G, M):
    # Flatten real and imaginary parts separately
    v_rho = np.concatenate([rho.full().real.flatten(), rho.full().imag.flatten()])
    v_G   = np.concatenate([G.full().real.flatten(), G.full().imag.flatten()])
    v_M0  = np.concatenate([M[0].full().real.flatten(), M[0].full().imag.flatten()])
    v_M1  = np.concatenate([M[1].full().real.flatten(), M[1].full().imag.flatten()])
    return np.concatenate([v_rho, v_G, v_M0, v_M1])

# Build training data
X = np.array([flatten_experiment(rho, G, M) for rho, G, M in dataset])
y = np.array([pfail_vector(rho, G, M) for rho, G, M in dataset])


In [7]:
print(np.shape(X))
print(np.shape(y))

(1215, 32)
(1215, 3)
